In [1]:
import twitter

# Go to http://twitter.com/apps/new to create an app and get these items
# See https://dev.twitter.com/docs/auth/oauth for more information on Twitter's OAuth implementation

CONSUMER_KEY = 'F9UuextodupQI1SM5MFk0dKJk'
CONSUMER_SECRET = 'Rnudn4IJggeM1LP0ysNfckOfutjc9NxbdRSuiFiAj5hivcdfwE'
OAUTH_TOKEN = '75511442-FBWYJK7RgUITFgAJ8soTvhvRj8bBvGXkjN5T1kO0Z'
OAUTH_TOKEN_SECRET = 'tMtURFmUKKfs8thsaNVW2n38JykzcXq8oPJUIUODlydO6'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(domain='api.twitter.com', 
                              api_version='1.1',
                              auth=auth
                             )

In [7]:
# With an authenticated twitter_api in existence, you can now use it to query Twitter resources as usual.
# However, the trends resource is cleaned up a bit in v1.1, so requests are a bit simpler than in the latest
# printing. See https://dev.twitter.com/docs/api/1.1/get/trends/place

# The Yahoo! Where On Earth ID for the entire world is 1
WORLD_WOE_ID = 1 
DK_WOE_ID = 23424796
# Prefix id with the underscore for query string parameterization.
# Without the underscore, it's appended to the URL itself
world_trends = twitter_api.trends.place(_id=DK_WOE_ID)
#print world_trends

In [8]:
#exercise 1.2
import json
print json.dumps(world_trends, indent=1)

[
 {
  "created_at": "2016-09-18T20:44:31Z", 
  "trends": [
   {
    "url": "http://twitter.com/search?q=%23MRSA", 
    "query": "%23MRSA", 
    "tweet_volume": null, 
    "name": "#MRSA", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23fcklive", 
    "query": "%23fcklive", 
    "tweet_volume": null, 
    "name": "#fcklive", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23rvlm16", 
    "query": "%23rvlm16", 
    "tweet_volume": null, 
    "name": "#rvlm16", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23obfcm", 
    "query": "%23obfcm", 
    "tweet_volume": null, 
    "name": "#obfcm", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23FCNSJE", 
    "query": "%23FCNSJE", 
    "tweet_volume": null, 
    "name": "#FCNSJE", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=S%C3%B8nderjyske", 
    "query": "S%C3%B8

In [27]:
#ex 1.3 print nicely json
for item in world_trends[0]["trends"]:
    print item["name"]
    counter += 1

#MRSA
#fcklive
#rvlm16
#obfcm
#FCNSJE
Sønderjyske
Tulle
Chelsea
Snapchat
Amager
Rusland
Trump
Danmarks
Esbjerg
#vuls16
#Brøndby
#MUFC
#cphhalf
#UNGA
#DHOpen
#1div
#SingaporeGP
#EuroRoad2016
#isdk
#sportfyn
#nfldk
#EnecoTour
#Paralympics
#UNIX
#HBeredskab


In [30]:
#ex 1.5
# Like all other APIs, search requests now require authentication and have a slightly different request and
# response format. See https://dev.twitter.com/docs/api/1.1/get/search/tweets

q = "#2030Now"
count = 100

search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']

# v1.1 of Twitter's API provides a value in the response for the next batch of results that needs to be parsed out
# and passed back in as keyword args if you want to retrieve more than one page. It appears in the 'search_metadata'
# field of the response object and has the following form:'?max_id=313519052523986943&q=NCAA&include_entities=1'
# The tweets themselves are encoded in the 'statuses' field of the response


# Here's how you would grab five more batches of results and collect the statuses as a list
for _ in range(5): 
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError, e: # No more results when next_results doesn't exist
        break

    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ]) # Create a dictionary from the query string params
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

In [31]:
print json.dumps(statuses[0:2], indent=1) # Only print a couple of tweets here in IPython Notebook

[
 {
  "contributors": null, 
  "truncated": false, 
  "text": "RT @AlexBPearson: Great conversation on #Refugee Children in Crisis with @kasujja, @RefugeesChief, &amp; @carolynsave. #2030Now https://t.co/wL\u2026", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 777614646492721152, 
  "favorite_count": 0, 
  "entities": {
   "symbols": [], 
   "user_mentions": [
    {
     "id": 21060994, 
     "indices": [
      3, 
      16
     ], 
     "id_str": "21060994", 
     "screen_name": "AlexBPearson", 
     "name": "Alex Pearson"
    }, 
    {
     "id": 15096735, 
     "indices": [
      73, 
      81
     ], 
     "id_str": "15096735", 
     "screen_name": "kasujja", 
     "name": "Alan Kasujja"
    }, 
    {
     "id": 4353234441, 
     "indices": [
      83, 
      97
     ], 
     "id_str": "4353234441", 
     "screen_name": "RefugeesChief", 
     "name": "Filippo Grandi"
    }, 
    {
     "id": 194140922, 
     "indices": [
      105, 
      117
     ], 
   

In [56]:
tweets = [ status['text'] for status in statuses ] #get text from tweets
retweets = [ status['retweet_count'] for status in statuses] #get retweets

zipped = zip(retweets,tweets) #join tweets and retweets tghter
most_popular_tweet = max(zipped) #find max 
print most_popular_tweet[1]

RT @ValaAfshar: If 100 people lived on earth🌍

1 person has 50% of money💵
13 with no clean drinking water💧
14 can't read📘 #2030Now 

https:…
